In [ ]:
from vespa.application import Vespa
from dotenv import load_dotenv  
load_dotenv()
import os

# Path to your security files
cert_path = "/Users/shivamjogdand/Desktop/Learning/DeepAgents/Report_Generation_Git/Report_Generation/certs/public.pem"
key_path = "/Users/shivamjogdand/Desktop/Learning/DeepAgents/Report_Generation_Git/Report_Generation/certs/private.pem"
endpoint_url = os.getenv("VESPA_ENDPOINT")

app = Vespa(
    url=endpoint_url,
    cert=cert_path,
    key=key_path
)

# table search 

In [ ]:
import re
from concurrent.futures import ThreadPoolExecutor
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel
from dotenv import load_dotenv
load_dotenv()
import os

# ======================================================
# 0. Models
# ======================================================

class TableResponse(BaseModel):
    is_financial_statement: bool 
    title: str | None = None
    raw_content: str | None = None 

# ======================================================
# 1. Gemini Setup
# ======================================================

TABLE_CLASSIFICATION_PROMPT = """
Analyze the following document page. 
DETERMINE if it contains a primary financial statement table:
- Income Statement / Comprehensive Income
- Balance Sheet / Financial Position
- Cash Flow Statement
- Statement of Changes in Equity

OUTPUT FORMAT (JSON ONLY):
{{
  "is_financial_statement": true/false,
  "title": "[Exact Statement Name from text]"
}}

INPUT:
Title: {page_title}
Content: {content}
"""

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", 
    temperature=0, 
    api_key=os.getenv("GOOGLE_API_KEY")
)

structured_model = gemini_llm.with_structured_output(
    schema=TableResponse.model_json_schema(),
    method="json_schema"
)

# ======================================================
# 2. Regex Helpers
# ======================================================

TABLE_REGEX = re.compile(r'(\|.+\|\n\|[-:\s|]+\|\n(?:\|.*\|\n?)*)', re.MULTILINE)

def is_table_present(content: str) -> bool:
    if not content: return False
    return len(TABLE_REGEX.findall(content)) > 0

# ======================================================
# 3. Vespa Logic (UNCHANGED)
# ======================================================

def get_page_numbers_with_tables(hits):
    pages = {
        h["fields"]["page_number"]
        for h in hits
        if is_table_present(h["fields"].get("content", ""))
    }
    print(f"[INFO] Initial pages with numeric tables: {sorted(pages)}")
    return pages

def fetch_specific_pages(app, tenant_id, file_id, page_list):
    if not page_list: return []
    pages_str = ", ".join(map(str, page_list))
    yql = (
        f"select title, page_number, content "
        f"from sources pefund.pefund "
        f"where tenant_id contains \"{tenant_id}\" "
        f"AND file_id contains \"{file_id}\" "
        f"AND page_number in ({pages_str})"
    )
    res = app.query(body={"yql": yql, "hits": len(page_list)})
    return res.hits if hasattr(res, "hits") else res.get("hits", [])

def get_complete_table_sequence(app, tenant_id, file_id, query):
    yql = (
        f"select title, page_number, content "
        f"from sources pefund.pefund "
        f"where tenant_id contains \"{tenant_id}\" "
        f"AND file_id contains \"{file_id}\" "
        f"AND (userQuery() or ({{targetHits:100}}nearestNeighbor(question_embeddings,q)))"
    )
    res = app.query({
        "yql": yql,
        "query": query,
        "input.query(q)": f"embed(e5-small-v2, \"{query}\")",
        "ranking": "default",
        "hits": 20
    })
    hits = res.hits if hasattr(res, "hits") else res.get("hits", [])
    confirmed = get_page_numbers_with_tables(hits)
    checked = set()
    while True:
        candidates = {
            n for p in confirmed
            for n in (p - 1, p + 1)
            if n > 0 and n not in confirmed and n not in checked
        }
        if not candidates: break
        new_hits = fetch_specific_pages(app, tenant_id, file_id, list(candidates))
        hit_map = {h["fields"]["page_number"]: h for h in new_hits}
        found = False
        for p in candidates:
            hit = hit_map.get(p)
            if not hit:
                checked.add(p)
                continue
            if is_table_present(hit["fields"].get("content", "")):
                confirmed.add(p)
                found = True
            else:
                checked.add(p)
        if not found: break
    final_pages = sorted(confirmed)
    print(f"[INFO] Final table pages: {final_pages}")
    final_hits = fetch_specific_pages(app, tenant_id, file_id, final_pages)
    final_hits.sort(key=lambda h: h["fields"]["page_number"])
    return final_hits

# ======================================================
# 4. Parallel Processing
# ======================================================

def process_single_hit(args):
    idx, hit = args
    content = hit["fields"].get("content", "")
    
    result = structured_model.invoke(
        TABLE_CLASSIFICATION_PROMPT.format(
            page_title=hit["fields"].get("title", ""),
            content=content
        )
    )

    parsed = TableResponse.model_validate(result)
    parsed.raw_content = content 

    if not parsed.is_financial_statement:
        return idx, None

    return idx, parsed

# ======================================================
# 6. Execute & Write to File
# ======================================================

TENANT_ID = "0f7a4b5f-a137-4a71-b518-1a04ba239b61"
FILE_ID = "9de6cf15-b315-4270-92c0-aa648b818949"
QUERY = "consolidated financial statements"
OUTPUT_FILE = "outputs/output_md_files/reconstructed_financial_tables_final_002.md"

hits = get_complete_table_sequence(app, TENANT_ID, FILE_ID, QUERY)
results = [None] * len(hits)

with ThreadPoolExecutor(max_workers=5) as pool:
    for idx, response in pool.map(process_single_hit, enumerate(hits)):
        results[idx] = response

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for r in results:
        if r and r.raw_content:
            # Title handling: Ensure ### prefix
            if r.title:
                clean_title = r.title.lstrip("# ").strip()
                f.write(f"### {clean_title}\n\n")
            
            f.write(r.raw_content + "\n\n")
            f.write("---\n\n")

print(f"[SUCCESS] Extraction completed. Data saved to {OUTPUT_FILE}")

[INFO] Initial pages with numeric tables: [171, 178, 238, 239, 240, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279]
[INFO] Final table pages: [171, 172, 173, 174, 175, 176, 178, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281]
[SUCCESS] Extraction completed. Data saved to outputs/output_md_files/reconstructed_financial_tables_final_002.md


# Question Generation

In [ ]:
import os
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

# ======================================================
# 1. Setup & Configuration
# ======================================================

# Define the headers to split on (matches our ### Title from previous step)
headers_to_split_on = [
    ("###", "Header_Title"),
]

# Initialize the Splitter
header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Initialize Gemini for Question Generation
gemini_pe_analyst = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro", 
    temperature=0.7, 
    api_key=os.getenv("GOOGLE_API_KEY")
)

class QuestionList(BaseModel):
    questions: List[str] = Field(description="List of due diligence questions")

# Structured output for clean parsing
structured_qa_model = gemini_pe_analyst.with_structured_output(QuestionList)

# ======================================================
# 2. Updated Managing Director PE Prompt
# ======================================================

PE_DUE_DILIGENCE_PROMPT = """
[SYSTEM OPERATING DIRECTIVE: ACT AS A SENIOR MANAGING DIRECTOR & HEAD OF DUE DILIGENCE]
You are conducting a final Investment Committee (IC) review for a multi-billion dollar acquisition. You are skeptical, intellectually aggressive, and focused on protecting Limited Partner (LP) capital.

**TARGET SEGMENT:** {header_title}
**FINANCIAL DATA SOURCE:** ---
{chunk_content}
---

**CORE OBJECTIVE:**
Generate 5-7 "Investment Committee Grade" questions. Your goal is to dismantle the management's narrative and uncover the "Truth behind the GAAP/IFRS numbers."

**STRATEGIC QUESTIONING PILLARS:**
1. **Quality of Earnings (QoE) & Accrual Integrity:** Identify where non-cash items or "one-time" adjustments might be masking a decline in core operational health. Look at Note 34 (Discontinued Operations) or Note 31 (Zakat/Tax) to see if tax releases are padding the bottom line.
2. **Capital Intensity & Asset Obsolescence:** If looking at the Statement of Financial Position, cross-reference PP&E (112B) against Depreciation (11.4B). Ask if the current "Repairs and Maintenance" (4.2B) is sufficient to maintain competitive parity or if there is a massive CAPEX "bow wave" coming.
3. **Liquidity & Structural Subordination:** Analyze the SAR 45.3B in Current Liabilities against the SAR 30.5B Cash position. Question the reliance on "Short-term investments" and "Trade payables" as a primary financing vehicle.
4. **Macro-Economic Sensitivity:** Specifically target the SAR/USD peg. Ask about the "Exchange difference on translation" (which showed a 1.2B loss in 2024) and how a potential de-pegging or regional instability would impact the LTM debt-to-equity ratio.
5. **Entity-Level Complexity:** Use the data from "Material Associates" or "Non-Controlling Interests" to ask about structural leakage. (e.g., "Why are we seeing high dividend outflows to NCI (2.6B) while the Parent's Retained Earnings are under pressure?").

**MANDATORY CONSTRAINTS:**
- NO generic questions (e.g., "Why did revenue change?").
- USE advanced financial metrics: ROIC, Free Cash Flow Conversion, Working Capital Intensity, and LTM Bridge Analysis.
- DO NOT be limited by the provided text; if the text suggests a risk (like a discontinued operation), ask about the strategic fallout even if the answer isn't in the chunk.
"""

# ======================================================
# 3. Processing & File Saving
# ======================================================

INPUT_FILE = "outputs/output_md_files/reconstructed_financial_tables_final_002.md"
QUESTIONS_OUTPUT_FILE = "outputs/output_md_files/pe_due_diligence_questions_pro.md"

def generate_pe_questions():
    if not os.path.exists(INPUT_FILE):
        print(f"[ERROR] Input file {INPUT_FILE} not found.")
        return

    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        md_content = f.read()

    # Split the document based on the ### Headers
    sections = header_splitter.split_text(md_content)
    
    all_qa_content = []

    print(f"[INFO] Processing {len(sections)} financial sections...")

    for section in sections:
        header = section.metadata.get("Header_Title", "Financial Statement Section")
        content = section.page_content

        # Generate Questions via Gemini using the MD Prompt
        formatted_prompt = PE_DUE_DILIGENCE_PROMPT.format(
            header_title=header,
            chunk_content=content
        )
        
        try:
            response = structured_qa_model.invoke(formatted_prompt)
            
            # Format for the output file
            section_output = f"## Due Diligence Questions: {header}\n\n"
            for i, q in enumerate(response.questions, 1):
                section_output += f"{i}. {q}\n"
            section_output += "\n---\n\n"
            
            all_qa_content.append(section_output)
            print(f"[SUCCESS] Generated questions for: {header}")
            
        except Exception as e:
            print(f"[ERROR] Failed for {header}: {e}")

    # Save to final file
    with open(QUESTIONS_OUTPUT_FILE, "w", encoding="utf-8") as f:
        f.writelines(all_qa_content)

    print(f"[FINISH] All questions saved to {QUESTIONS_OUTPUT_FILE}")

generate_pe_questions()

[INFO] Processing 6 financial sections...
[SUCCESS] Generated questions for: CONSOLIDATED STATEMENT OF FINANCIAL POSITION
[SUCCESS] Generated questions for: CONSOLIDATED STATEMENT OF INCOME
[SUCCESS] Generated questions for: CONSOLIDATED STATEMENT OF COMPREHENSIVE INCOME
[SUCCESS] Generated questions for: CONSOLIDATED STATEMENT OF CHANGES IN EQUITY
[SUCCESS] Generated questions for: CONSOLIDATED STATEMENT OF CASH FLOWS
[SUCCESS] Generated questions for: CONSOLIDATED STATEMENT OF CASH FLOWS (CONTINUED)
[FINISH] All questions saved to outputs/output_md_files/pe_due_diligence_questions_pro.md


# /questionnaire endpoint

In [ ]:
import os
import uuid
import httpx
import json
import time
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from typing import List

# ======================================================
# 1. Configuration & Persona Setup
# ======================================================

# Endpoint Configuration
API_URL = "http://localhost:8000/api/questionnaire" 
TENANT_ID = "d90887dc-391e-430c-921f-6d3f86bf8a81" 
INPUT_FILE = "outputs/output_md_files/sample_table.md"
JSON_OUTPUT_FILE = "outputs/output_md_files/pe_due_diligence_results.json"

# Model Setup
gemini_pe_analyst = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.7, 
    api_key=os.getenv("GOOGLE_API_KEY")
)

class QuestionList(BaseModel):
    questions: List[str] = Field(description="List of due diligence questions")

structured_qa_model = gemini_pe_analyst.with_structured_output(QuestionList)

# ======================================================
# 2. Robust PE Managing Director Prompt
# ======================================================

PE_DUE_DILIGENCE_PROMPT = """
[SYSTEM OPERATING DIRECTIVE: ACT AS A SENIOR MANAGING DIRECTOR & HEAD OF DUE DILIGENCE]
You are conducting a final Investment Committee (IC) review for a multi-billion dollar acquisition. You are skeptical, intellectually aggressive, and focused on protecting Limited Partner (LP) capital.

**TARGET SEGMENT:** {header_title}
**FINANCIAL DATA SOURCE:** ---
{chunk_content}
---

**CORE OBJECTIVE:**
Generate 5-7 "Investment Committee Grade" questions. Your goal is to dismantle the management's narrative and uncover the "Truth behind the GAAP/IFRS numbers."

**STRATEGIC QUESTIONING PILLARS:**
1. **Quality of Earnings (QoE) & Accrual Integrity:** Identify where non-cash items or "one-time" adjustments might be masking a decline in core operational health. Look at Note 34 (Discontinued Operations) or Note 31 (Zakat/Tax) to see if tax releases are padding the bottom line.
2. **Capital Intensity & Asset Obsolescence:** If looking at the Statement of Financial Position, cross-reference PP&E (112B) against Depreciation (11.4B). Ask if the current "Repairs and Maintenance" (4.2B) is sufficient to maintain competitive parity or if there is a massive CAPEX "bow wave" coming.
3. **Liquidity & Structural Subordination:** Analyze the SAR 45.3B in Current Liabilities against the SAR 30.5B Cash position. Question the reliance on "Short-term investments" and "Trade payables" as a primary financing vehicle.
4. **Macro-Economic Sensitivity:** Specifically target the SAR/USD peg. Ask about the "Exchange difference on translation" (which showed a 1.2B loss in 2024) and how a potential de-pegging or regional instability would impact the LTM debt-to-equity ratio.
5. **Entity-Level Complexity:** Use the data from "Material Associates" or "Non-Controlling Interests" to ask about structural leakage. (e.g., "Why are we seeing high dividend outflows to NCI (2.6B) while the Parent's Retained Earnings are under pressure?").

**MANDATORY CONSTRAINTS:**
- NO generic questions (e.g., "Why did revenue change?").
- USE advanced financial metrics: ROIC, Free Cash Flow Conversion, Working Capital Intensity, and LTM Bridge Analysis.
- DO NOT be limited by the provided text; if the text suggests a risk (like a discontinued operation), ask about the strategic fallout even if the answer isn't in the chunk.
"""

# ======================================================
# 3. Execution Engine
# ======================================================

def run_sequential_pipeline():
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found.")
        return

    # 1. Read and Split Content
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        md_content = f.read()

    headers_to_split_on = [("###", "Header_Title")]
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    sections = splitter.split_text(md_content)

    final_report_data = []
    print(f"[INFO] Found {len(sections)} financial sections. Starting sequential processing...")

    # 2. Process each section one by one
    for section in sections:
        header = section.metadata.get("Header_Title", "Financial Section")
        print(f"\n{'='*60}\nSECTION: {header}\n{'='*60}")

        section_results = {
            "section_title": header,
            "timestamp": time.ctime(),
            "due_diligence_inquiries": []
        }

        # Generate Questions
        formatted_prompt = PE_DUE_DILIGENCE_PROMPT.format(
            header_title=header,
            chunk_content=section.page_content
        )

        try:
            response = structured_qa_model.invoke(formatted_prompt)
            questions = response.questions
            print(f"[LLM] Generated {len(questions)} high-stakes questions.")
        except Exception as e:
            print(f"[LLM ERROR] Failed to generate questions for {header}: {e}")
            continue

        # 3. Hit API for each question sequentially
        for i, q in enumerate(questions, 1):
            print(f"\n[QUESTION {i}]: {q}")

            payload = {
                "query": q,
                "description": f"Senior PE Due Diligence analysis for {header}. Strategic risk evaluation.",
                "request_id": str(uuid.uuid4()),
                "web_search": False,
                "response_type": "informative",
                "tenant_id": TENANT_ID
            }

            with httpx.Client(timeout=180.0) as client:
                try:
                    api_resp = client.post(API_URL, json=payload)
                    if api_resp.status_code == 200:
                        full_result = api_resp.json()
                        print(f"[API RESPONSE]: SUCCESS - Request ID: {payload['request_id']}")
                        
                        # Properly capture complete JSON response for each inquiry
                        section_results["due_diligence_inquiries"].append({
                            "question": q,
                            "api_response": full_result
                        })
                    else:
                        print(f"[API ERROR]: Status {api_resp.status_code} - {api_resp.text}")
                except Exception as e:
                    print(f"[API CONNECTION ERROR]: {e}")

        final_report_data.append(section_results)

    # 4. Save entire object as properly formatted JSON
    with open(JSON_OUTPUT_FILE, "w", encoding="utf-8") as json_file:
        json.dump(final_report_data, json_file, indent=4, ensure_ascii=False)

    print(f"\n[SUCCESS] Full report saved to {JSON_OUTPUT_FILE}")


run_sequential_pipeline()

[INFO] Found 1 financial sections. Starting sequential processing...

SECTION: CONSOLIDATED STATEMENT OF FINANCIAL POSITION
[LLM] Generated 5 high-stakes questions.

[QUESTION 1]: The balance sheet reflects a significant reduction in 'Assets held for sale' (from SAR 15.4B to SAR 3.6B) and associated liabilities, alongside a sharp decline in 'Zakat payable' (from SAR 1.4B to SAR 0.1B) and 'Current Provisions' (from SAR 1.1B to SAR 0.1B). What is the LTM Free Cash Flow Conversion adjusted for the impact of these asset disposals and potential one-time P&L benefits from provision/Zakat releases? Specifically, provide a bridge showing how these non-recurring items influenced reported operating cash flow and the material increase in 'Other current assets and receivables' (SAR 5B+).
[API RESPONSE]: SUCCESS - Request ID: 8fa6a014-f004-47bb-bb75-805e61a615cb

[QUESTION 2]: Total non-current assets, particularly Property, Plant and Equipment (Note 7) and Investments in associates and joint ventu

# /chat endpoint

In [ ]:
import os
import uuid
import httpx
import json
import time
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from typing import List

# ======================================================
# 1. Configuration & Persona Setup
# ======================================================

# Endpoint Configuration
API_URL = "http://localhost:8000/api/chat" 
TENANT_ID = "0f7a4b5f-a137-4a71-b518-1a04ba239b61" # Updated to match your chat payload
INPUT_FILE = "outputs/output_md_files/sample_table.md"
JSON_OUTPUT_FILE = "outputs/output_md_files/pe_due_diligence_results_chat.json"

# Model Setup
gemini_pe_analyst = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.7, 
    api_key=os.getenv("GOOGLE_API_KEY")
)

class QuestionList(BaseModel):
    questions: List[str] = Field(description="List of due diligence questions")

structured_qa_model = gemini_pe_analyst.with_structured_output(QuestionList)

# ======================================================
# 2. Robust PE Managing Director Prompt
# ======================================================

PE_DUE_DILIGENCE_PROMPT = """
[SYSTEM OPERATING DIRECTIVE: ACT AS A SENIOR MANAGING DIRECTOR & HEAD OF DUE DILIGENCE]
You are conducting a final Investment Committee (IC) review for a multi-billion dollar acquisition. You are skeptical, intellectually aggressive, and focused on protecting Limited Partner (LP) capital.

**TARGET SEGMENT:** {header_title}
**FINANCIAL DATA SOURCE:** ---
{chunk_content}
---

**CORE OBJECTIVE:**
Generate 5-7 "Investment Committee Grade" questions. Your goal is to dismantle the management's narrative and uncover the "Truth behind the GAAP/IFRS numbers."

**STRATEGIC QUESTIONING PILLARS:**
1. **Quality of Earnings (QoE) & Accrual Integrity:** Identify where non-cash items or "one-time" adjustments might be masking a decline in core operational health.
2. **Capital Intensity & Asset Obsolescence:** Question if maintenance CAPEX is sufficient to avoid operational obsolescence.
3. **Liquidity & Structural Subordination:** Analyze Current Liabilities against Cash positions.
4. **Macro-Economic Sensitivity:** Specifically target the SAR/USD peg and translation losses.
5. **Entity-Level Complexity:** Look at Material Associates or Non-Controlling Interests for structural leakage.

**MANDATORY CONSTRAINTS:**
- NO generic questions.
- USE advanced financial metrics: ROIC, Free Cash Flow Conversion, Working Capital Intensity.
- DO NOT be limited by the provided text; ask about strategic fallout and hidden risks.
"""

# ======================================================
# 3. Execution Engine
# ======================================================

def run_chat_sequential_pipeline():
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found.")
        return

    # 1. Read and Split Content
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        md_content = f.read()

    headers_to_split_on = [("###", "Header_Title")]
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    sections = splitter.split_text(md_content)

    final_report_data = []
    session_id = str(uuid.uuid4()) # Persistent session for this entire run
    print(f"[INFO] Found {len(sections)} financial sections. Starting sequential Chat processing...")

    # 2. Process each section one by one
    for section in sections:
        header = section.metadata.get("Header_Title", "Financial Section")
        print(f"\n{'='*60}\nSECTION: {header}\n{'='*60}")

        section_results = {
            "section_title": header,
            "timestamp": time.ctime(),
            "chat_inquiries": []
        }

        # Generate Questions
        formatted_prompt = PE_DUE_DILIGENCE_PROMPT.format(
            header_title=header,
            chunk_content=section.page_content
        )

        try:
            response = structured_qa_model.invoke(formatted_prompt)
            questions = response.questions
            print(f"[LLM] Generated {len(questions)} high-stakes questions.")
        except Exception as e:
            print(f"[LLM ERROR] Failed to generate questions for {header}: {e}")
            continue

        # 3. Hit /chat API for each question sequentially
        for i, q in enumerate(questions, 1):
            print(f"\n[QUESTION {i}]: {q}")

            # Construct Chat Payload
            payload = {
                "query": q,
                "session_id": session_id,
                "message_id": str(i),
                "web_search": False,
                "conversations": [],
                "tenant_ids": [TENANT_ID]
            }

            with httpx.Client(timeout=180.0) as client:
                try:
                    api_resp = client.post(API_URL, json=payload)
                    if api_resp.status_code == 200:
                        full_result = api_resp.json()
                        print(f"[API RESPONSE]: SUCCESS - Session ID: {session_id}")
                        
                        # Properly capture complete JSON response for each inquiry
                        section_results["chat_inquiries"].append({
                            "question": q,
                            "api_response": full_result
                        })
                    else:
                        print(f"[API ERROR]: Status {api_resp.status_code} - {api_resp.text}")
                except Exception as e:
                    print(f"[API CONNECTION ERROR]: {e}")

        final_report_data.append(section_results)

    # 4. Save entire object as properly formatted JSON
    with open(JSON_OUTPUT_FILE, "w", encoding="utf-8") as json_file:
        json.dump(final_report_data, json_file, indent=4, ensure_ascii=False)

    print(f"\n[SUCCESS] Full Chat report saved to {JSON_OUTPUT_FILE}")


run_chat_sequential_pipeline()

[INFO] Found 1 financial sections. Starting sequential Chat processing...

SECTION: CONSOLIDATED STATEMENT OF FINANCIAL POSITION
[LLM] Generated 5 high-stakes questions.

[QUESTION 1]: The significant decrease in 'Assets held for sale' from SAR 15.4 billion to SAR 3.6 billion, coupled with the corresponding elimination of 'Liabilities directly associated with assets held for sale' (SAR 5.7 billion), suggests substantial divestitures. What was the net gain or loss on these disposals, how much of it was recognized as 'one-off' or non-operating income, and what is the pro-forma Quality of Earnings, Free Cash Flow Conversion, and ROIC of the *retained* core business, isolating the impact of these asset sales?
[API RESPONSE]: SUCCESS - Session ID: 71a45270-34e3-4dae-bbd0-28cc2b90b2c6

[QUESTION 2]: Net Property, Plant & Equipment and Intangible Assets have both shown declines. What is the company's actual maintenance CAPEX relative to depreciation and amortization over the last three years,